In [24]:
import sys, pathlib
import joblib
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from pathlib import Path
from src import config as cfg
PROJECT_ROOT = pathlib.Path().resolve().parent.parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))
from src import config as cfg

# ── Cargar datos
data = joblib.load(cfg.DATA / "processed" / "lstm_data.pkl")
X, y = data["X"], data["y"]
fechas = pd.to_datetime(data["dates"])
print("✅ Datos cargados:", X.shape, y.shape)


✅ Datos cargados: (3239, 60, 40) (3239, 40)


In [26]:
# ── División temporal por fechas
train_mask = fechas < "2019-01-01"
val_mask   = (fechas >= "2019-01-01") & (fechas < "2021-01-01")
test_mask  = fechas >= "2021-01-01"

X_train, y_train = X[train_mask], y[train_mask]
X_val, y_val     = X[val_mask], y[val_mask]
X_test, y_test   = X[test_mask], y[test_mask]

print("🔹 Train:", X_train.shape)
print("🔹 Val:  ", X_val.shape)
print("🔹 Test: ", X_test.shape)


print("🗓️ Rango fechas:")
print("Train:", fechas[train_mask].min(), "→", fechas[train_mask].max())
print("Val:  ", fechas[val_mask].min(), "→", fechas[val_mask].max())
print("Test: ", fechas[test_mask].min(), "→", fechas[test_mask].max())

🔹 Train: (1609, 60, 40)
🔹 Val:   (505, 60, 40)
🔹 Test:  (1125, 60, 40)
🗓️ Rango fechas:
Train: 2012-08-08 00:00:00 → 2018-12-31 00:00:00
Val:   2019-01-02 00:00:00 → 2020-12-31 00:00:00
Test:  2021-01-04 00:00:00 → 2025-06-26 00:00:00


In [28]:
# ── Escalado
scaler_X = StandardScaler()
X_train_2d = X_train.reshape(-1, X.shape[2])
X_train_scaled = scaler_X.fit_transform(X_train_2d).reshape(X_train.shape)

X_val_scaled  = scaler_X.transform(X_val.reshape(-1, X.shape[2])).reshape(X_val.shape)
X_test_scaled = scaler_X.transform(X_test.reshape(-1, X.shape[2])).reshape(X_test.shape)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled   = scaler_y.transform(y_val)
y_test_scaled  = scaler_y.transform(y_test)

# Guardar escaladores
joblib.dump(scaler_X, cfg.MODELS / "scaler_X_lstm.pkl")
joblib.dump(scaler_y, cfg.MODELS / "scaler_y_lstm.pkl")
print("✅ Escaladores guardados.")

✅ Escaladores guardados.


In [30]:
# ── Definir modelo
model = models.Sequential([
    layers.Input(shape=(cfg.WINDOW, X.shape[2])),
    layers.LSTM(64, return_sequences=True),
    layers.Dropout(0.2),
    layers.LSTM(32),
    layers.Dense(y.shape[1])
])
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss="mse")
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 60, 64)         │        26,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 60, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 40)             │         1,320 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,616 (158.66 KB)

 Trainable params: 40,616 (158.66 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
# ── Entrenar
early_stop = EarlyStopping(patience=5, restore_best_weights=True)
history = model.fit(
    X_train_scaled, y_train_scaled,
    validation_data=(X_val_scaled, y_val_scaled),
    epochs=50,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 5s 47ms/step - loss: 0.9615 - val_loss: 2.6193
Epoch 2/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - loss: 0.9535 - val_loss: 2.6210
Epoch 3/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - loss: 1.0342 - val_loss: 2.6280
Epoch 4/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 52ms/step - loss: 1.0130 - val_loss: 2.6320
Epoch 5/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - loss: 0.9514 - val_loss: 2.6468
Epoch 6/50
51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step - loss: 0.9652 - val_loss: 2.6394


In [34]:
joblib.dump(history.history, cfg.RESULT / "history_lstm.pkl")
print("✅ Histórico de entrenamiento guardado.")

# ── Guardar modelo
Path(cfg.MODELS).mkdir(parents=True, exist_ok=True)
model.save(cfg.MODELS / "lstm_t1.keras")
print("✅ Modelo guardado.")

# ── Evaluar en test
y_pred = model.predict(X_test_scaled)
rmse = np.sqrt(((y_test_scaled - y_pred)**2).mean(axis=0))
rmse_mean = rmse.mean()
print("📉 RMSE medio:", rmse_mean)
joblib.dump(rmse_mean, cfg.RESULT / "rmse_lstm.pkl")
print("✅ RMSE guardado.")

✅ Histórico de entrenamiento guardado.
✅ Modelo guardado.
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
📉 RMSE medio: 1.2399235
✅ RMSE guardado.
